In [8]:
import sqlite3
import pandas as pd

## Importing Categorized data

In [9]:
csv_path_2320 = './data/categorizedData/TransaccionesBAC.Transacciones_Tarjeta_Colones_2320.csv'
csv_path_3193 = './data/categorizedData/TransaccionesBAC.Transacciones_Tarjeta_Dolares_3193.csv'

csv_2320 = pd.read_csv(csv_path_2320, encoding='latin1', delimiter=';')
csv_3193 = pd.read_csv(csv_path_3193, encoding='latin1', delimiter=';')

## Updating the tables with the new data categorized

In [10]:
db_path = './data/dbs/TransaccionesBAC.db'

conn = sqlite3.connect(db_path)

def update_table_with_csv_reference(table_name, csv_data, conn):
    cursor = conn.cursor()
    
    cursor.execute(f"PRAGMA table_info({table_name})")
    columns = [info[1] for info in cursor.fetchall()]
    if "Category" not in columns:
        cursor.execute(f"ALTER TABLE {table_name} ADD COLUMN Category TEXT")
    if "SubCategory" not in columns:
        cursor.execute(f"ALTER TABLE {table_name} ADD COLUMN SubCategory TEXT")
    
    for _, row in csv_data.iterrows():
        cursor.execute(f"""
            UPDATE {table_name}
            SET Category = ?, SubCategory = ?
            WHERE Transaction_Reference = ?
        """, (row['Category'], row['SubCategory'], row['Transaction Reference']))
    
    conn.commit()
    cursor.close()

update_table_with_csv_reference("Transacciones_Tarjeta_Debito_2320", csv_2320, conn)
update_table_with_csv_reference("Transacciones_Tarjeta_Debito_3193", csv_3193, conn)

conn.close()

## Data Consulting

In [11]:
conn = sqlite3.connect(db_path)

cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

cursor.close()

In [12]:
dataframes = {}

for table in tables:
    table_name = table[0]
    if table_name != "sqlite_sequence":
        query = f"SELECT * FROM {table_name}"
        df = pd.read_sql_query(query, conn)
        dataframes[table_name] = df

conn.close()

#dataframes.keys()

In [13]:
debito_2320 = dataframes['Transacciones_Tarjeta_Debito_2320']
debito_3193 = dataframes['Transacciones_Tarjeta_Debito_3193']

credito_0243 = dataframes['Transacciones_Tarjeta_Credito_0243']
credito_3959 = dataframes['Transacciones_Tarjeta_Credito_3959']